In [2]:
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.metrics import f1_score, precision_score, precision_recall_curve, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from scipy.stats.mstats import winsorize
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor


In [3]:
#Import the data 
# read all tables 
customer_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
engagement_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
transactions_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

In [4]:
# deal with gender null 
# deal with age null
customer_final.dropna(subset=['gender', 'age'], inplace=True)

In [5]:
customer_final['join_date'] = pd.to_datetime(customer_final['join_date'])
customer_final['last_purchase_date'] = pd.to_datetime(customer_final['last_purchase_date'])
transactions_final['transaction_date'] = pd.to_datetime(transactions_final['transaction_date'])
marketing_final['campaign_date'] = pd.to_datetime(marketing_final['campaign_date'])

In [7]:
transactions_agg = transactions_final.groupby('customer_id').aggregate({'transaction_id':'count','transaction_amount':'sum'})

In [8]:
marketing_agg = marketing_final.groupby('customer_id').size().reset_index(name='number_of_campaigns')
marketing_agg.set_index('customer_id', inplace=True)


In [9]:
engagement_final.set_index('customer_id', inplace=True)
customer_final.set_index('customer_id', inplace=True)

In [10]:
joint_data = customer_final.join(engagement_final).join(transactions_agg).join(marketing_agg)

In [11]:
joint_data

,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,number_of_campaigns
customer_id,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,4
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,4
6,2022-01-01,2023-02-10,55.0,Female,North Richardfort,74,59,7,17,10254.44,3
8,2023-02-09,2023-08-09,68.0,Male,Marquezton,60,22,7,15,11710.62,3
9,2022-01-24,2023-08-15,68.0,Female,West Franciscobury,49,43,21,22,10382.96,2
...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,119,47,16,10,5498.20,3
9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,3,33,14,12,5848.30,1
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,1


In [12]:
joint_data['customer_lifetime'] = (joint_data['last_purchase_date'] - joint_data['join_date']).dt.days

In [13]:
# Winsorize  to handle zeros and extreme values
joint_data['customer_lifetime'] = winsorize(joint_data['customer_lifetime'], limits=[0.01, 0.01])

In [14]:
joint_data['normalized_clv'] = joint_data['transaction_amount'] / joint_data['customer_lifetime']

In [15]:
joint_data

,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,number_of_campaigns,customer_lifetime,normalized_clv
customer_id,,,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,4,118,29.741356
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,4,243,32.406091
6,2022-01-01,2023-02-10,55.0,Female,North Richardfort,74,59,7,17,10254.44,3,405,25.319605
8,2023-02-09,2023-08-09,68.0,Male,Marquezton,60,22,7,15,11710.62,3,181,64.699558
9,2022-01-24,2023-08-15,68.0,Female,West Franciscobury,49,43,21,22,10382.96,2,568,18.279859
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,119,47,16,10,5498.20,3,240,22.909167
9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,3,33,14,12,5848.30,1,200,29.241500
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,1,135,25.949111


In [16]:
zero_lifetime_customers = joint_data[joint_data['customer_lifetime'] == 0]

# Display the count
zero_lifetime_count = zero_lifetime_customers.shape[0]
zero_lifetime_count

0

In [17]:
# Feature selection
features = ['age', 'gender','number_of_site_visits', 'number_of_emails_opened',
            'number_of_clicks', 'number_of_campaigns', 'transaction_id']
target = 'normalized_clv'

In [19]:
# One-hot encode the 'gender' column
joint_data = pd.get_dummies(joint_data, columns=['gender'], drop_first=True)

In [21]:
# Convert to integer if necessary
joint_data['gender_Male'] = joint_data['gender_Male'].astype(int)

In [22]:
# Adjusted feature selection
features = ['age', 'number_of_site_visits', 'number_of_emails_opened',
            'number_of_clicks', 'number_of_campaigns', 'transaction_id', 'gender_Male']
target = 'normalized_clv'

In [23]:
# Scale numerical features
scaler = MinMaxScaler()
joint_data[features] = scaler.fit_transform(joint_data[features])

In [25]:
# Define the feature set and the target variable
X = joint_data[features]
y = joint_data[target]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)

In [27]:
# Model selection and training
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [28]:
y_pred_lr = lr_model.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

In [29]:
rmse = mean_squared_error(y_test, y_pred_lr, squared=False)

In [30]:
linear_performance = {'MSE': mse_lr, 'R2': r2_lr, 'RMSE': rmse}

# Display model performance
linear_performance

{'MSE': 368681.4993638256, 'R2': 0.0661044208014081, 'RMSE': 607.1914849236817}